In [1]:
import csv
file = open('train_data.csv')
csvreader = csv.reader(file)

header = next(csvreader)
print(f'file header: {header}')

rows = []
for row in csvreader:
        rows.append(row)

rows[0]

file header: ['id', 'q', 'r', 's', "q'", "r'", '', 'total no.: 7987']


['8',
 '"It can go both ways . We all doubt . It is what you do with it that matters ."',
 '"True ."',
 'AGREE',
 '"It can go both ways . We all doubt . It is what you do with it that matters ."',
 '"True ."',
 '',
 '']

In [2]:
import pandas as pd
X = pd.DataFrame(data=rows, columns=header)
Q = X.copy().drop(columns=['r', 's', "r'", '', "total no.: 7987"]).to_dict('records')
R = X.copy().drop(columns=['q', 's', "q'", '', "total no.: 7987"]).to_dict('records')

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

2022-11-09 08:03:09.031251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def words_to_tags(raw_data, statement, short):

    data = []

    for idx in range(len(raw_data)):

        obj = {}
        tags = []

        content = tokenizer(raw_data[idx].get(statement), is_split_into_words=False)
        content_token = tokenizer.convert_ids_to_tokens(content["input_ids"])
        label = tokenizer(raw_data[idx].get(short), is_split_into_words=False)
        label_token = tokenizer.convert_ids_to_tokens(label["input_ids"])

        idxForLabel=0

        # print(f'content: {content_token}')
        # print(f'label: {label_token}')

        for token in content_token:
            if (token == '[CLS]' or token == '[SEP]'):
                content_token.remove(token)
        
        for token in label_token:
            if (token == '[CLS]' or token == '[SEP]'):
                label_token.remove(token)

        for token in content_token:

            if (idxForLabel < len(label_token) and token==label_token[idxForLabel]):
                idxForLabel += 1
                tags.append(1)
            else:
                tags.append(0)
        
        obj['id'] = raw_data[idx].get('id')
        obj['tokens'] = content_token
        obj['tags'] = tags

        data.append(obj)
    
    return data


In [24]:
def tokenize_and_align_labels(examples):
    output = examples
    for idx in range(len(examples)):
        content = tokenizer(examples[idx]['tokens'], truncation=True, is_split_into_words=True)
        output[idx]["attention_mask"]=content["attention_mask"]
        output[idx]["input_ids"]=content["input_ids"]
        labels = []
        for i, label in enumerate(examples[idx]['tags']):
            arr = examples[idx]['tags']
            word_ids = content.word_ids()  # Map tokens to their respective word.
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:  # Set the special tokens to -100.
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                    label_ids.append(arr[word_idx])
                else:
                    label_ids.append(-100)
                previous_word_idx = word_idx
            output[idx]["labels"]=label_ids
    return output

In [27]:
Q = X.copy().drop(columns=['r', 's', "r'", '', "total no.: 7987"]).to_dict('records')
Q = words_to_tags(Q, "q", "q'")

In [ ]:
Q[2]

In [28]:
Q_ = tokenize_and_align_labels(Q)

In [ ]:
print(Q_[0].keys())
print(Q_[6]['input_ids'])
print(Q_[6]['attention_mask'])
print(Q_[6]['labels'])
print(len(Q_[6]['input_ids']))
print(len(Q_[6]['attention_mask']))
print(len(Q_[6]['labels']))

In [34]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [36]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [41]:
# Split the training data set and testing data set
len(Q_) # 38346

38346

In [43]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Q_[0:100],
    eval_dataset=Q_[101:110],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 21
  Number of trainable parameters = 66364418


  0%|          | 0/21 [00:00<?, ?it/s]

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, tokens, tags. If id, tokens, tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, tokens, tags. If id, tokens, tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.3345896303653717, 'eval_runtime': 1.5661, 'eval_samples_per_second': 5.747, 'eval_steps_per_second': 0.639, 'epoch': 1.0}


...

***** Running Evaluation *****
  Num examples = 9
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, tokens, tags. If id, tokens, tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.


  0%|          | 0/1 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.31831175088882446, 'eval_runtime': 1.7216, 'eval_samples_per_second': 5.228, 'eval_steps_per_second': 0.581, 'epoch': 3.0}
{'train_runtime': 503.064, 'train_samples_per_second': 0.596, 'train_steps_per_second': 0.042, 'train_loss': 0.37268749872843426, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=0.37268749872843426, metrics={'train_runtime': 503.064, 'train_samples_per_second': 0.596, 'train_steps_per_second': 0.042, 'train_loss': 0.37268749872843426, 'epoch': 3.0})

In [ ]:
trainer.predict()